# Build Data Frame for Toronto

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes
import folium

from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
Source = BeautifulSoup(website_url,'lxml')
Table = Source.find('table',{'class':'wikitable sortable'})
headers = "Postal Code, Borough, Neighborhood"

table1=""
for tr in Table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
    
file = open("toronto.csv", "wb")
file.write(bytes(table1,encoding="ascii", errors = "ignore"))

df = pd.read_csv('toronto.csv', header = None)
df.columns = ["Postal Code", "Borough", "Neighborhood"]

UnassignedNames = df[ df['Borough'] == 'Not assigned'].index
df.drop(UnassignedNames , inplace=True)
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']

CombinedPostalCode = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_1 = CombinedPostalCode.reset_index()
df_1.head()

!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_location = pd.read_csv('Toronto_long_lat_data.csv')
df_location.head()

df_2 = pd.merge(df_1,df_location[['Postal Code','Latitude', 'Longitude']],on='Postal Code')

In [3]:
df_2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:
CLIENT_ID = 'UGCUAOA02AWVH0URHOV3KVCYHE2VV4X3O5NMY2HPLJDZNJNR' # your Foursquare ID
CLIENT_SECRET = 'QP2HFQWSUFJNZOGBEZ01UKASPV4YHISVEPQAQ4FL5K40S4CO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UGCUAOA02AWVH0URHOV3KVCYHE2VV4X3O5NMY2HPLJDZNJNR
CLIENT_SECRET:QP2HFQWSUFJNZOGBEZ01UKASPV4YHISVEPQAQ4FL5K40S4CO


In [158]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )
toronto_venues.head()

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,GreenWin pool,43.756232,-79.333842,Pool
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [109]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [161]:
corr = toronto_grouped.corr()
#corr.style.background_gradient(cmap='coolwarm')

In [164]:
corr.dtypes

Yoga Studio                      float64
Accessories Store                float64
Afghan Restaurant                float64
Airport                          float64
Airport Food Court               float64
Airport Gate                     float64
Airport Lounge                   float64
Airport Service                  float64
Airport Terminal                 float64
American Restaurant              float64
Antique Shop                     float64
Aquarium                         float64
Art Gallery                      float64
Art Museum                       float64
Arts & Crafts Store              float64
Asian Restaurant                 float64
Athletics & Sports               float64
Auto Garage                      float64
Auto Workshop                    float64
BBQ Joint                        float64
Baby Store                       float64
Bagel Shop                       float64
Bakery                           float64
Bank                             float64
Bar             

In [176]:
#PS = pd.DataFrame()
#PS['Correlation'] = corr['Pet Store']
#PS['Venue Type'] = corr[0:'Pet Store']
PS = corr['Pet Store']
PS_df = pd.DataFrame()
PS_df['Correlation'] = PS[:]
PS_df.head()

,Correlation
Yoga Studio,0.147182
Accessories Store,-0.027335
Afghan Restaurant,-0.027335
Airport,-0.031725
Airport Food Court,-0.027335


In [113]:
PS_sorted=PS_df.sort_values('Correlation', ascending=False)
PS_sorted.head(10)

,Correlation
Pet Store,1.000000
Latin American Restaurant,0.633725
Indian Restaurant,0.578725
Thrift / Vintage Store,0.508594
Vietnamese Restaurant,0.482325
Furniture / Home Store,0.335193
Bridal Shop,0.271313
Frozen Yogurt Shop,0.271313
Gastropub,0.263341
Fish & Chips Shop,0.248131


(276, 1)

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

NVS = neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Hotel,Restaurant,Burger Joint,Asian Restaurant,Thai Restaurant
1,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Pharmacy,Gluten-free Restaurant,Electronics Store
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Sandwich Place,Pub,Dance Studio,Coffee Shop,Gym,Eastern European Restaurant,Dumpling Restaurant


In [153]:
Neigh_PS_weighted = pd.DataFrame()
Neigh_PS_weighted['Neighborhood'] = toronto_grouped['Neighborhood']
Neigh_PS_weighted['weighted'] = toronto_grouped['Latin American Restaurant']*0.633725 + toronto_grouped['Indian Restaurant']*0.578725 + toronto_grouped['Thrift / Vintage Store']*0.508594 + toronto_grouped['Vietnamese Restaurant']*0.482325 + toronto_grouped['Furniture / Home Store']*0.335193
Neigh_PS_weighted.head()

,Neighborhood,weighted
0,"Adelaide, King, Richmond",0.012124
1,Agincourt,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000
4,"Alderwood, Long Branch",0.000000


In [154]:
Neigh_sorted=Neigh_PS_weighted.sort_values('weighted', ascending=False)
Neigh_sorted.head()

,Neighborhood,weighted
34,"Dorset Park, Scarborough Town Centre, Wexford ...",0.389661
62,"Lawrence Heights, Lawrence Manor",0.068127
89,Thorncliffe Park,0.068085
70,"Northwood Park, York University",0.067039
60,"Kingsway Park South West, Mimico NW, The Queen...",0.036328


In [ ]:
NVS